# Table of Contents

&emsp;[<a id="#vocab" /> <font color='yellow' size=5> Vocab </font>](#%3Ca-id%3D%22%23vocab%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Vocab-%3C/font%3E)<br>
&emsp;&emsp;[1. Comparison](#1.-Comparison)<br>
&emsp;&emsp;[2. Intensifier](#2.-Intensifier)<br><br>
&emsp;[<a id="Taxonomy" /> <font color='yellow' size=5> Taxonomy </font>](#%3Ca-id%3D%22Taxonomy%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Taxonomy-%3C/font%3E)<br>
&emsp;&emsp;[1. Size, Shape, Color, Age, Material](#1.-Size%2C-Shape%2C-Color%2C-Age%2C-Material)<br>
&emsp;&emsp;[2. Professions vs nationalities](#2.-Professions-vs-nationalities)<br>
&emsp;&emsp;[3. Animal vs Vehicles - Example1](#3.-Animal-vs-Vehicles---Example1)<br>
&emsp;&emsp;[4. Animal vs Vehicles - Example2](#4.-Animal-vs-Vehicles---Example2)<br>
&emsp;&emsp;[5. Synonyms](#5.-Synonyms)<br>
&emsp;&emsp;[6. Antonyms](#6.-Antonyms)<br>
&emsp;&emsp;[7. Antonyms Comparison](#7.-Antonyms-Comparison)<br><br>
&emsp;[<a id="Temporal" /> <font color='yellow' size=5> Temporal </font>](#%3Ca-id%3D%22Temporal%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Temporal-%3C/font%3E)<br>
&emsp;&emsp;[1. Change in profession](#1.-Change-in-profession)<br>
&emsp;&emsp;[2. Understand time difference](#2.-Understand-time-difference)<br><br>
&emsp;[<a id="Negation" /> <font color='yellow' size=5> Negation </font>](#%3Ca-id%3D%22Negation%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Negation-%3C/font%3E)<br>
&emsp;&emsp;[1. Negation in Context](#1.-Negation-in-Context)<br>
&emsp;&emsp;[3. Negation in Question](#3.-Negation-in-Question)<br><br>
&emsp;[<a id="Coref" /> <font color='yellow' size=5> Coref </font>](#%3Ca-id%3D%22Coref%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Coref-%3C/font%3E)<br>
&emsp;&emsp;[1. He/she coref](#1.-He/she-coref)<br>
&emsp;&emsp;[2. His/Her coref](#2.-His/Her-coref)<br>
&emsp;&emsp;[3. Former and Latter](#3.-Former-and-Latter)<br>

In [1]:
%load_ext autoreload
%autoreload 2
import checklist.editor
import munch
import itertools
from checklist.test_suite import TestSuite
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.perturb import Perturb
from helper import get_finetuned_electra_predictor, format_squad_with_context,show_example, crossproduct, export_suite_to_jsonl, get_summary,display_and_export_mdtable, clean, expect_squad,generate_dataset

In [2]:
suite = TestSuite()
editor = checklist.editor.Editor()
model_predictor = get_finetuned_electra_predictor()
expect_squad = Expect.single(expect_squad)

/opt/anaconda3/envs/nlp-proj/lib/python3.10/site-packages/iso639/iso639.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


## <a id="#vocab" /> <font color='yellow'> Vocab </font>

### 1. Comparison

In [3]:
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]


t_comparison = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    ),(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    ),
    ],
    labels = ['{first_name1}','{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=1000,
    save=True
)

In [4]:
show_example(t_comparison, n=1)

  [1] ('Andrew is higher than Stephen.', 'Who is less high?') , pred: Andrew , label: Stephen
  [2] ('Andrew is higher than Stephen.', 'Who is higher?') , pred: Andrew , label: Andrew
---


In [5]:
name = 'Comparisons'
description = 'A is COMP than B. Who is more / less COMP?'
test = MFT(**t_comparison, name=name, description=description, capability='Vocabulary')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1996 examples
Test cases:      998
Fails (rate):    998 (100.0%)

Example fails:
C: Anne is greater than Donna.
Q: Who is less great?
A: Donna
P: Anne


----


### 2. Intensifier

In [6]:
state = editor.suggest('John is very {mask} about the project.')[:20]
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

t_intensifier = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=1000,
    save=True
    ))

/opt/anaconda3/envs/nlp-proj/lib/python3.10/site-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


In [7]:
show_example(t_intensifier)

  [1] ('Dorothy is really enthusiastic about the project. Jimmy is enthusiastic about the project.', 'Who is most enthusiastic about the project?') , pred: Jimmy , label: Dorothy
  [2] ('Dorothy is really enthusiastic about the project. Jimmy is enthusiastic about the project.', 'Who is least enthusiastic about the project?') , pred: Jimmy , label: Jimmy
  [3] ('Jimmy is enthusiastic about the project. Dorothy is really enthusiastic about the project.', 'Who is most enthusiastic about the project?') , pred: Dorothy , label: Dorothy
  [4] ('Jimmy is enthusiastic about the project. Dorothy is really enthusiastic about the project.', 'Who is least enthusiastic about the project?') , pred: Dorothy , label: Jimmy
  [5] ('Dorothy is enthusiastic about the project. Jimmy is somewhat enthusiastic about the project.', 'Who is most enthusiastic about the project?') , pred: Jimmy , label: Dorothy
  [6] ('Dorothy is enthusiastic about the project. Jimmy is somewhat enthusiastic about the project.'

In [8]:
name = 'Intensifiers'
desc = '(very, super, extremely) and reducers (somewhat, kinda, etc)?'
test = MFT(**t_intensifier, name=name, description=desc, capability='Vocabulary')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context, n_per_testcase=2)
suite.add(test)

Predicting 11952 examples
Test cases:      996
Fails (rate):    996 (100.0%)

Example fails:
C: Suzanne is highly vocal about the project. Christopher is vocal about the project.
Q: Who is most vocal about the project?
A: Suzanne
P: Christopher

C: Christopher is vocal about the project. Suzanne is highly vocal about the project.
Q: Who is least vocal about the project?
A: Christopher
P: Suzanne


----


## <a id="Taxonomy" /> <font color='yellow'> Taxonomy </font>

### 1. Size, Shape, Color, Age, Material

In [9]:
order = ['size', 'shape', 'age', 'color']
props = []
properties = {
    'color' : ['red', 'blue','yellow', 'green', 'pink', 'white', 'black', 'orange', 'grey', 'purple', 'brown'],
    'size' : ['big', 'small', 'tiny', 'enormous'],
    'age' : ['old', 'new'],
    'shape' : ['round', 'oval', 'square', 'triangular'],
    'material' : ['iron', 'wooden', 'ceramic', 'glass', 'stone']
}
objects = ['box', 'clock', 'table', 'object', 'toy', 'painting', 'sculpture', 'thing', 'figure']

for i in range(len(order)):
    for j in range(i + 1, len(order)):
        p1, p2 = order[i], order[j]
        for v1, v2 in itertools.product(properties[p1], properties[p2]):
            props.append(munch.Munch({
                'p1': p1,
                'p2': p2,
                'v1': v1,
                'v2': v2,
            }))
            
t_properties = crossproduct(editor.template(
    {
        'contexts': [
            'There is {a:p.v1} {p.v2} {obj} in the room.',
            'There is {a:obj} in the room. The {obj} is {p.v1} and {p.v2}.',
        ],
        'qas': [
            (
                'What {p.p1} is the {obj}?',
                '{p.v1}'
            ), 
            (
                'What {p.p2} is the {obj}?',
                '{p.v2}'
            ), 
            
        ]
    },
    obj=objects,
    p=props,
    remove_duplicates=True,
    nsamples=1000,
    save=True
    ))


In [10]:
show_example(t_properties, n=2)

  [1] ('There is a square grey table in the room.', 'What shape is the table?') , pred: square grey , label: square
  [2] ('There is a square grey table in the room.', 'What color is the table?') , pred: grey , label: grey
  [3] ('There is a table in the room. The table is square and grey.', 'What shape is the table?') , pred: square and grey , label: square
  [4] ('There is a table in the room. The table is square and grey.', 'What color is the table?') , pred: grey , label: grey
---
  [1] ('There is a tiny grey table in the room.', 'What size is the table?') , pred: grey , label: tiny
  [2] ('There is a tiny grey table in the room.', 'What color is the table?') , pred: grey , label: grey
  [3] ('There is a table in the room. The table is tiny and grey.', 'What size is the table?') , pred: tiny and grey , label: tiny
  [4] ('There is a table in the room. The table is tiny and grey.', 'What color is the table?') , pred: grey , label: grey
---


In [11]:
name = 'Properites'
desc = 'size, shape, age, color'
test = MFT(**t_properties, name=name, description=desc, capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 4000 examples
Test cases:      1000
Fails (rate):    999 (99.9%)

Example fails:
C: There is a square grey painting in the room.
Q: What shape is the painting?
A: square
P: square grey

C: There is a painting in the room. The painting is square and grey.
Q: What shape is the painting?
A: square
P: square and grey


----


### 2. Professions vs nationalities

In [12]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')

In [13]:
t_profession_nationtionality = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=1000,
    save=True,
    ))

In [14]:
show_example(t_profession_nationtionality, n=1)

  [1] ('Patricia is an American organizer.', "What is Patricia's job?") , pred: organizer , label: organizer
  [2] ('Patricia is an American organizer.', "What is Patricia's nationality?") , pred: American , label: American
  [3] ('Patricia is an organizer. Patricia is American.', "What is Patricia's job?") , pred: organizer , label: organizer
  [4] ('Patricia is an organizer. Patricia is American.', "What is Patricia's nationality?") , pred: American , label: American
  [5] ('Patricia is American. Patricia is an organizer.', "What is Patricia's job?") , pred: organizer , label: organizer
  [6] ('Patricia is American. Patricia is an organizer.', "What is Patricia's nationality?") , pred: American , label: American
  [7] ('Patricia is American and an organizer.', "What is Patricia's job?") , pred: organizer , label: organizer
  [8] ('Patricia is American and an organizer.', "What is Patricia's nationality?") , pred: American , label: American
  [9] ('Patricia is an organizer and America

In [15]:
name = 'Profession vs nationality'
test = MFT(**t_profession_nationtionality, name=name, expect=expect_squad, description='The ability to distinguish between professions and nationalities',  capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 10000 examples
Test cases:      1000
Fails (rate):    521 (52.1%)

Example fails:
C: Michelle is an entrepreneur and Bangladeshi.
Q: What is Michelle's job?
A: entrepreneur
P: entrepreneur and Bangladeshi


----


### 3. Animal vs Vehicles - Example1

In [16]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t_animal_vehicles = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=1000,
    save=True
    ))

In [17]:
show_example(t_animal_vehicles)

  [1] ('Alan has a fish and a train.', 'What animal does Alan have?') , pred: a fish and a train , label: fish
  [2] ('Alan has a fish and a train.', 'What vehicle does Alan have?') , pred: a fish and a train , label: train
  [3] ('Alan has a train and a fish.', 'What animal does Alan have?') , pred: a train and a fish , label: fish
  [4] ('Alan has a train and a fish.', 'What vehicle does Alan have?') , pred: a train , label: train
---


In [18]:
name = 'Animal vs Vehicle'
test = MFT(**t_animal_vehicles, name=name, description='The ability to distinguish between animals and vehicles by asking about the object (animal or vehicle)', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)

Predicting 4000 examples
Test cases:      1000
Fails (rate):    991 (99.1%)

Example fails:
C: Francis has a hamster and a minivan.
Q: What vehicle does Francis have?
A: minivan
P: hamster and a minivan

C: Francis has a minivan and a hamster.
Q: What animal does Francis have?
A: hamster
P: minivan and a hamster


----


### 4. Animal vs Vehicles - Example2

In [19]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t_animals_vehicles_2 = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} bought {a:animal}. {first_name2} bought {a:vehicle}.',
            '{first_name2} bought {a:vehicle}. {first_name} bought {a:animal}.',
        ],
        'qas': [
            (
                'Who bought an animal?',
                '{first_name}'
            ), 
            (
                'Who bought a vehicle?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=1000,
    save=True
    ))


In [20]:
show_example(t_animals_vehicles_2)

  [1] ('Jill bought a lizard. Lynn bought a van.', 'Who bought an animal?') , pred: Jill , label: Jill
  [2] ('Jill bought a lizard. Lynn bought a van.', 'Who bought a vehicle?') , pred: Lynn , label: Lynn
  [3] ('Lynn bought a van. Jill bought a lizard.', 'Who bought an animal?') , pred: Jill , label: Jill
  [4] ('Lynn bought a van. Jill bought a lizard.', 'Who bought a vehicle?') , pred: Lynn , label: Lynn
---


In [21]:

name = 'Animal vs Vehicle v2'
test = MFT(**t_animals_vehicles_2, name=name, description='The ability to distinguish between animals and vehicles by asking about who bougth the object', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)

Predicting 3988 examples
Test cases:      997
Fails (rate):    534 (53.6%)

Example fails:
C: Catherine bought a firetruck. Larry bought a snake.
Q: Who bought a vehicle?
A: Catherine
P: Catherine bought a firetruck. Larry


----


### 5. Synonyms 

In [22]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

t_synonym = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[0]}. {first_name2} is very {s2[0]}.',
            '{first_name2} is very {s2[0]}. {first_name} is very {s1[0]}.',
        ],
        'qas': [
            (
                'Who is {s1[1]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[1]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=500,
    save=True
   ))
t_synonym += crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[1]}. {first_name2} is very {s2[1]}.',
            '{first_name2} is very {s2[1]}. {first_name} is very {s1[1]}.',
        ],
        'qas': [
            (
                'Who is {s1[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[0]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=500,
    save=True
    )) 

In [23]:
show_example(t_synonym)

  [1] ('Donald is very scared. Michelle is very humble.', 'Who is frightened?') , pred: Donald , label: Donald
  [2] ('Donald is very scared. Michelle is very humble.', 'Who is modest?') , pred: Michelle , label: Michelle
  [3] ('Michelle is very humble. Donald is very scared.', 'Who is frightened?') , pred: Donald , label: Donald
  [4] ('Michelle is very humble. Donald is very scared.', 'Who is modest?') , pred: Michelle , label: Michelle
---


In [24]:
name = 'Synonyms'
test = MFT(**t_synonym, name=name, description='The ability to distinguish between synonyms', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3592 examples
Test cases:      898
Fails (rate):    63 (7.0%)

Example fails:
C: Suzanne is very vocal. Stephen is very angry.
Q: Who is outspoken?
A: Suzanne
P: Stephen


----


### 6. Antonyms

In [25]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))

In [26]:
t_antonymns = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )
            
        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=1000,
    save=True
    ))

In [27]:
show_example(t_antonymns)

  [1] ('Dave is smarter than Sam.', 'Who is dumber?') , pred: Dave , label: Sam
  [2] ('Dave is smarter than Sam.', 'Who is smarter?') , pred: Dave , label: Dave
  [3] ('Sam is dumber than Dave.', 'Who is dumber?') , pred: Sam , label: Sam
  [4] ('Sam is dumber than Dave.', 'Who is smarter?') , pred: Sam , label: Dave
---


In [28]:
name = 'Antonyms'
test = MFT(**t_antonymns, name=name, description='A is COMP than B. Who is antonym(COMP)? B', capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3984 examples
Test cases:      996
Fails (rate):    996 (100.0%)

Example fails:
C: Louis is taller than Anne.
Q: Who is shorter?
A: Anne
P: Louis

C: Anne is shorter than Louis.
Q: Who is taller?
A: Louis
P: Anne


----


### 7. Antonyms Comparison

In [29]:
antonym_adjs = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]

t_antonymns_compare = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is more {a[0]} than {first_name1}.',
            '{first_name1} is more {a[1]} than {first_name}.',
            '{first_name} is less {a[1]} than {first_name1}.',
            '{first_name1} is less {a[0]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is more {a[0]}?',
                '{first_name}',
            ),
            (
                'Who is less {a[0]}?',
                '{first_name1}',
            ),
            (
                'Who is more {a[1]}?',
                '{first_name1}',
            ),
            (
                'Who is less {a[1]}?',
                '{first_name}',
            ),
        ]
        ,
    },
    a = antonym_adjs,
    remove_duplicates=True,
    nsamples=1000,
    save=True
    ))

In [30]:
show_example(t_antonymns_compare)

  [1] ('Fiona is more humble than Sharon.', 'Who is more humble?') , pred: Fiona , label: Fiona
  [2] ('Fiona is more humble than Sharon.', 'Who is less humble?') , pred: Fiona , label: Sharon
  [3] ('Fiona is more humble than Sharon.', 'Who is more proud?') , pred: Fiona , label: Sharon
  [4] ('Fiona is more humble than Sharon.', 'Who is less proud?') , pred: Fiona , label: Fiona
  [5] ('Sharon is more proud than Fiona.', 'Who is more humble?') , pred: Sharon , label: Fiona
  [6] ('Sharon is more proud than Fiona.', 'Who is less humble?') , pred: Sharon , label: Sharon
  [7] ('Sharon is more proud than Fiona.', 'Who is more proud?') , pred: Sharon , label: Sharon
  [8] ('Sharon is more proud than Fiona.', 'Who is less proud?') , pred: Sharon , label: Fiona
  [9] ('Fiona is less proud than Sharon.', 'Who is more humble?') , pred: Fiona , label: Fiona
  [10] ('Fiona is less proud than Sharon.', 'Who is less humble?') , pred: Fiona , label: Sharon
  [11] ('Fiona is less proud than Sharon

In [31]:
description = 'A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.'
name= 'Antonyms Comparison'
test = MFT(**t_antonymns_compare, name=name, description=description, capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 15920 examples
Test cases:      995
Fails (rate):    995 (100.0%)

Example fails:
C: Bruce is more negative than Alice.
Q: Who is less negative?
A: Alice
P: Bruce

C: Bruce is more negative than Alice.
Q: Who is more positive?
A: Alice
P: Bruce

C: Alice is more positive than Bruce.
Q: Who is more negative?
A: Bruce
P: Alice


----


## <a id="Temporal" /> <font color='yellow'> Temporal </font>

### 1. Change in profession

In [32]:
t_profession = crossproduct(editor.template(
    {
        'contexts': [
            'Both {first_name} and {first_name2} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
            'Both {first_name2} and {first_name} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
        ],
        'qas': [
            (
                'Who is {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=1000,
    ))


In [33]:
show_example(t_profession)

  [1] ('Both Arthur and Walter were historians, but there was a change in Arthur, who is now an economist.', 'Who is an economist?') , pred: Arthur , label: Arthur
  [2] ('Both Walter and Arthur were historians, but there was a change in Arthur, who is now an economist.', 'Who is an economist?') , pred: Arthur , label: Arthur
---


In [34]:
name = 'Change in profession'
test = MFT(**t_profession, expect=expect_squad, capability='Temporal', name=name, description='The ability to detect a change in profession over time' )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1930 examples
Test cases:      965
Fails (rate):    14 (1.5%)

Example fails:
C: Both William and Alexander were educators, but there was a change in Alexander, who is now an author.
Q: Who is an author?
A: Alexander
P: empty


----


### 2. Understand time difference
e.g before, after

In [35]:
t_time_difference = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} became a {prof} before {first_name2} did.',
            '{first_name2} became a {prof} after {first_name} did.',
        ],
        'qas': [
            (
                'Who became a {prof} first?',
                '{first_name}'
            ), 
            (
                'Who became a {prof} last?',
                '{first_name2}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=1000,
    ))

In [36]:
show_example(t_time_difference)

  [1] ('Alexander became a waitress before Melissa did.', 'Who became a waitress first?') , pred: Alexander , label: Alexander
  [2] ('Alexander became a waitress before Melissa did.', 'Who became a waitress last?') , pred: Alexander , label: Melissa
  [3] ('Melissa became a waitress after Alexander did.', 'Who became a waitress first?') , pred: Melissa , label: Alexander
  [4] ('Melissa became a waitress after Alexander did.', 'Who became a waitress last?') , pred: Melissa , label: Melissa
---


In [37]:
description = 'Understanding before / after -> first / last.'
name="Time Difference"
test = MFT(**t_time_difference, expect=expect_squad, capability='Temporal', name=name, description=description )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3968 examples
Test cases:      992
Fails (rate):    992 (100.0%)

Example fails:
C: Annie became a entrepreneur before Katherine did.
Q: Who became a entrepreneur last?
A: Katherine
P: Annie

C: Katherine became a entrepreneur after Annie did.
Q: Who became a entrepreneur first?
A: Annie
P: Katherine

C: Katherine became a entrepreneur after Annie did.
Q: Who became a entrepreneur last?
A: Katherine
P: empty


----


## <a id="Negation" /> <font color='yellow'> Negation </font>

### 1. Negation in Context

In [38]:
t_context_negation = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is not {a:prof}. {first_name2} is.',
            '{first_name2} is {a:prof}. {first_name} is not.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=1000,
    ))

In [39]:
show_example(t_context_negation)

  [1] ('Lynn is not a historian. Rachel is.', 'Who is a historian?') , pred: Lynn , label: Rachel
  [2] ('Lynn is not a historian. Rachel is.', 'Who is not a historian?') , pred: Lynn , label: Lynn
  [3] ('Rachel is a historian. Lynn is not.', 'Who is a historian?') , pred: Rachel , label: Rachel
  [4] ('Rachel is a historian. Lynn is not.', 'Who is not a historian?') , pred: Lynn , label: Lynn
---


In [40]:

name = 'Negation in context, may or may not be in question'
test = MFT(**t_context_negation, expect=expect_squad, capability='Negation', name=name, description='Ability to handle negation in context, whether or not it appears in the question' )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3984 examples
Test cases:      996
Fails (rate):    980 (98.4%)

Example fails:
C: Jean is not a historian. Steven is.
Q: Who is a historian?
A: Steven
P: Jean


----


### 3. Negation in Question

In [41]:
t_question_negation = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:prof}. {first_name2} is {a:prof2}.',
            '{first_name2} is {a:prof2}. {first_name} is {a:prof}.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is {a:prof2}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=1000,
    ))


In [42]:

show_example(t_question_negation)

  [1] ('Johnny is an analyst. Patricia is an artist.', 'Who is an analyst?') , pred: Johnny , label: Johnny
  [2] ('Johnny is an analyst. Patricia is an artist.', 'Who is not an analyst?') , pred: Johnny , label: Patricia
  [3] ('Johnny is an analyst. Patricia is an artist.', 'Who is an artist?') , pred: Patricia , label: Patricia
  [4] ('Johnny is an analyst. Patricia is an artist.', 'Who is not an artist?') , pred: Patricia , label: Johnny
  [5] ('Patricia is an artist. Johnny is an analyst.', 'Who is an analyst?') , pred: Johnny , label: Johnny
  [6] ('Patricia is an artist. Johnny is an analyst.', 'Who is not an analyst?') , pred: Johnny , label: Patricia
  [7] ('Patricia is an artist. Johnny is an analyst.', 'Who is an artist?') , pred: Patricia , label: Patricia
  [8] ('Patricia is an artist. Johnny is an analyst.', 'Who is not an artist?') , pred: Patricia , label: Johnny
---


In [43]:
name = 'Negation in question only.'
test = MFT(**t_question_negation, expect=expect_squad, capability='Negation', name=name, description='The ability to handle negation when it appears only in the question' )
test.run(model_predictor)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 7728 examples
Test cases:      966
Fails (rate):    966 (100.0%)

Example fails:
C: Amanda is an actress. Paul is a photographer.
Q: Who is not an actress?
A: Paul
P: Amanda

C: Amanda is an actress. Paul is a photographer.
Q: Who is not a photographer?
A: Amanda
P: Paul

C: Paul is a photographer. Amanda is an actress.
Q: Who is not an actress?
A: Paul
P: Amanda


----
C: Charlotte is an organizer. Alexandra is a photographer.
Q: Who is not an organizer?
A: Alexandra
P: Charlotte

C: Charlotte is an organizer. Alexandra is a photographer.
Q: Who is not a photographer?
A: Charlotte
P: Alexandra

C: Alexandra is a photographer. Charlotte is an organizer.
Q: Who is not an organizer?
A: Alexandra
P: Charlotte


----
C: Carolyn is an architect. Andrea is a reporter.
Q: Who is not an architect?
A: Andrea
P: Carolyn

C: Carolyn is an architect. Andrea is a reporter.
Q: Who is not a reporter?
A: Carolyn
P: Andrea

C: Andrea is a reporter. Carolyn is an architect.
Q: Who is not an a

## <a id="Coref" /> <font color='yellow'> Coref </font>

### 1. He/she coref

In [44]:
if 'actress' in professions:
    professions.remove('actress')

t_he_she_coref = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{female} and {male} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{male} and {female} are friends. She is {a:prof2}, and he is {a:prof1}.',
            '{female} and {male} are friends. She is {a:prof2}, and he is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{male}'
            ), 
            (
                'Who is {a:prof2}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=1000,
    ))


In [45]:
show_example(t_he_she_coref)

  [1] ('Adam and Barbara are friends. He is an assistant, and she is an actor.', 'Who is an assistant?') , pred: Adam and Barbara , label: Adam
  [2] ('Adam and Barbara are friends. He is an assistant, and she is an actor.', 'Who is an actor?') , pred: Adam and Barbara , label: Barbara
  [3] ('Barbara and Adam are friends. He is an assistant, and she is an actor.', 'Who is an assistant?') , pred: Barbara and Adam , label: Adam
  [4] ('Barbara and Adam are friends. He is an assistant, and she is an actor.', 'Who is an actor?') , pred: Barbara and Adam , label: Barbara
  [5] ('Adam and Barbara are friends. She is an actor, and he is an assistant.', 'Who is an assistant?') , pred: Adam and Barbara , label: Adam
  [6] ('Adam and Barbara are friends. She is an actor, and he is an assistant.', 'Who is an actor?') , pred: Adam and Barbara , label: Barbara
  [7] ('Barbara and Adam are friends. She is an actor, and he is an assistant.', 'Who is an assistant?') , pred: Barbara and Adam , label: 

In [46]:
name = 'Basic coref, he / she'
test = MFT(**t_he_she_coref, expect=expect_squad, name=name, description='Ability to handle basic coreference with he and she given names in context', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 7864 examples
Test cases:      983
Fails (rate):    983 (100.0%)

Example fails:
C: Brian and Sarah are friends. He is an architect, and she is a photographer.
Q: Who is an architect?
A: Brian
P: Brian and Sarah

C: Brian and Sarah are friends. He is an architect, and she is a photographer.
Q: Who is a photographer?
A: Sarah
P: Brian and Sarah

C: Sarah and Brian are friends. He is an architect, and she is a photographer.
Q: Who is an architect?
A: Brian
P: Sarah and Brian


----


### 2. His/Her coref

In [47]:
t_his_her = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. His mom is {a:prof}.',
            '{female} and {male} are friends. His mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{male}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
t_his_her += crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. Her mom is {a:prof}.',
            '{female} and {male} are friends. Her mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))


In [48]:
show_example(t_his_her)

  [1] ('Roger and Carolyn are friends. His mom is a waitress.', 'Whose mom is a waitress?') , pred: Roger and Carolyn , label: Roger
  [2] ('Carolyn and Roger are friends. His mom is a waitress.', 'Whose mom is a waitress?') , pred: Carolyn and Roger , label: Roger
---


In [49]:
name = 'Basic coref, his / her'
test = MFT(**t_his_her, expect=expect_squad, name=name, description='Ability to handle basic coreference with his and her given names in context', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 2000 examples
Test cases:      1000
Fails (rate):    1000 (100.0%)

Example fails:
C: Anthony and Amanda are friends. His mom is a secretary.
Q: Whose mom is a secretary?
A: Anthony
P: Anthony and Amanda

C: Amanda and Anthony are friends. His mom is a secretary.
Q: Whose mom is a secretary?
A: Anthony
P: Amanda and Anthony


----


### 3. Former and Latter

In [50]:
t_former_latter = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} and {first_name2} are friends. The former is {a:prof1}.',
            '{first_name2} and {first_name} are friends. The latter is {a:prof1}.',
            '{first_name} and {first_name2} are friends. The former is {a:prof1} and the latter is {a:prof2}.',
            '{first_name2} and {first_name} are friends. The former is {a:prof2} and the latter is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=1000,
    save=True
    ))


In [51]:
show_example(t_former_latter)

  [1] ('Roy and Matthew are friends. The former is a model.', 'Who is a model?') , pred: Roy and Matthew , label: Roy
  [2] ('Matthew and Roy are friends. The latter is a model.', 'Who is a model?') , pred: Matthew and Roy , label: Roy
  [3] ('Roy and Matthew are friends. The former is a model and the latter is an entrepreneur.', 'Who is a model?') , pred: Roy and Matthew are friends. The former is a model and the latter is an entrepreneur , label: Roy
  [4] ('Matthew and Roy are friends. The former is an entrepreneur and the latter is a model.', 'Who is a model?') , pred: Matthew and Roy , label: Roy
---


In [52]:
name = 'Former / Latter'
test = MFT(**t_former_latter, expect=expect_squad, name=name, description='Ability to handle coreference with former and latter given names in context', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3848 examples
Test cases:      962
Fails (rate):    962 (100.0%)

Example fails:
C: Margaret and Al are friends. The former is a photographer.
Q: Who is a photographer?
A: Margaret
P: Margaret and Al

C: Al and Margaret are friends. The latter is a photographer.
Q: Who is a photographer?
A: Margaret
P: Al and Margaret

C: Margaret and Al are friends. The former is a photographer and the latter is an investor.
Q: Who is a photographer?
A: Margaret
P: Margaret and Al are friends. The former is a photographer and the latter is an investor


----


In [53]:
df_suite=get_summary(suite)

In [54]:
display_and_export_mdtable(df_suite, do_export=False)

| Test Name                                          |   Total Cases |   Example Per Case |   Failures | Failure Rate   |
|----------------------------------------------------|---------------|--------------------|------------|----------------|
| Comparisons                                        |           998 |                  2 |        998 | 100.00%        |
| Intensifiers                                       |           996 |                 12 |        996 | 100.00%        |
| Properites                                         |          1000 |                  4 |        999 | 99.90%         |
| Profession vs nationality                          |          1000 |                 10 |        521 | 52.10%         |
| Animal vs Vehicle                                  |          1000 |                  4 |        991 | 99.10%         |
| Animal vs Vehicle v2                               |           997 |                  4 |        534 | 53.56%         |
| Synonyms                                           |           898 |                  4 |         63 | 7.02%          |
| Antonyms                                           |           996 |                  4 |        996 | 100.00%        |
| Antonyms Comparison                                |           995 |                 16 |        995 | 100.00%        |
| Change in profession                               |           965 |                  2 |         14 | 1.45%          |
| Time Difference                                    |           992 |                  4 |        992 | 100.00%        |
| Negation in context, may or may not be in question |           996 |                  4 |        980 | 98.39%         |
| Negation in question only.                         |           966 |                  8 |        966 | 100.00%        |
| Basic coref, he / she                              |           983 |                  8 |        983 | 100.00%        |
| Basic coref, his / her                             |          1000 |                  2 |       1000 | 100.00%        |
| Former / Latter                                    |           962 |                  4 |        962 | 100.00%        |

In [55]:
generate_dataset(suite=suite,output_folder="ChecklistData", train_ratio=0.7)

✅ Saved, 90754 rows to ChecklistData/temp_combined.jsonl.
🔒 Generated 90754 unique IDs (collision-free)
📊 Splitting each test separately with 70.0%/20.0%/10.0% train/test/val ratios:
   Comparisons: 699/200/99 groups (70.0%/20.0%/9.9% examples)
   Intensifiers: 697/199/100 groups (70.0%/20.0%/10.0% examples)
   Properites: 700/200/100 groups (70.0%/20.0%/10.0% examples)
   Profession vs nationality: 700/200/100 groups (70.0%/20.0%/10.0% examples)
   Animal vs Vehicle: 700/200/100 groups (70.0%/20.0%/10.0% examples)
   Animal vs Vehicle v2: 698/199/100 groups (70.0%/20.0%/10.0% examples)
   Synonyms: 629/180/89 groups (70.0%/20.0%/9.9% examples)
   Antonyms: 697/199/100 groups (70.0%/20.0%/10.0% examples)
   Antonyms Comparison: 696/199/100 groups (69.9%/20.0%/10.1% examples)
   Change in profession: 676/193/96 groups (70.1%/20.0%/9.9% examples)
   Time Difference: 694/198/100 groups (70.0%/20.0%/10.1% examples)
   Negation in context, may or may not be in question: 697/199/100 groups (

([{'id': '9dcd02c283344c1da9722de6',
   'title': 'Comparisons',
   'context': 'Ruth is higher than Ralph.',
   'question': 'Who is less high?',
   'answers': {'text': ['Ralph'], 'answer_start': [20]},
   'pred': 'Ruth',
   'test_details': {'test_type': 'MFT',
    'capability': 'Vocabulary',
    'description': 'A is COMP than B. Who is more / less COMP?',
    'name': 'Comparisons',
    'testcase_id': 816,
    'qa_pair_index': 0,
    'total_qa_pairs': 2,
    'expect_function': 'expect'}},
  {'id': '88c909def1744e72930c590e',
   'title': 'Comparisons',
   'context': 'Ruth is higher than Ralph.',
   'question': 'Who is higher?',
   'answers': {'text': ['Ruth'], 'answer_start': [0]},
   'pred': 'Ruth',
   'test_details': {'test_type': 'MFT',
    'capability': 'Vocabulary',
    'description': 'A is COMP than B. Who is more / less COMP?',
    'name': 'Comparisons',
    'testcase_id': 816,
    'qa_pair_index': 1,
    'total_qa_pairs': 2,
    'expect_function': 'expect'}},
  {'id': '6e8d1eb6db0